In [16]:
import pandas as pd

In [17]:
AP = pd.read_csv('AirPassengers.csv')
print(AP.shape)
print(AP.head())

(144, 2)
      Date  Passengers
0  1949-01         112
1  1949-02         118
2  1949-03         132
3  1949-04         129
4  1949-05         121


In [18]:
print(AP.describe())

       Passengers
count  144.000000
mean   280.298611
std    119.966317
min    104.000000
25%    180.000000
50%    265.500000
75%    360.500000
max    622.000000


In [19]:
print(AP.tail())

        Date  Passengers
139  1960-08         606
140  1960-09         508
141  1960-10         461
142  1960-11         390
143  1960-12         432


In [22]:
AP['Date'] = pd.to_datetime(AP['Date'])
AP.head()

,Date,Passengers
0,1949-01-01,112
1,1949-02-01,118
2,1949-03-01,132
3,1949-04-01,129
4,1949-05-01,121


In [23]:
AP.isnull().sum()

Date          0
Passengers    0
dtype: int64